In [77]:
# Import libraries
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests

In [2]:
#Query url
url = "https://collegedunia.com/university/25787-lovely-professional-university-lpu-jalandhar/reviews"

In [3]:
"""
This piece of code not worked since api is not showing usual behaviour. 
If page doesn't exits it returns deafault page
"""

# i = 0 
# content = []
# while i < 100:
#     try:         
#         output = requests.get(url + "&page-{}".format(i))
#         soup = BeautifulSoup(output.text)
#         review_content = soup.findAll("div", { "class" : "content_data" })
#     except:
#         break
#     content.extend(review_content)

"\nThis piece of code not worked since api is not showing usual behaviour. \nIf page doesn't exits it returns deafault page\n"

In [4]:
# Scrap all review data available
output = requests.get(url)
soup = BeautifulSoup(output.text)
content = soup.findAll("div", { "class" : "content_data" })

### Total Positive/Negative votes for the college

In [28]:
# In order to classify positive and negative votes we are checking rating by each individual if it is greater than
# Mean value then we called it as positive and negative in opposite case.
total_positive = 0
total_negative = 0
mean_rating = soup.find("span", {"class" : "major_data"}).contents[0]
for rc in content:
    author_rating = rc.find("span", {"class" : "rating"}).string
    if author_rating >= mean_rating :
        total_positive += 1
    elif author_rating < mean_rating :
        total_negative += 1
    else :
        raise ValueError()

print "Total Positive votes :" + str(total_positive)
print "Total Negative votes :" + str(total_negative)

Total Positive votes :8
Total Negative votes :2


### Percentage of "review authors" in the following categories : “College students”, “Alumni"

In [32]:
# Calculate percentage of current students and alumni based on keywords available in the result
college_students = 0
alumni = 0
total_reviews = len(content)
unknown = 0
for rc in content:
    author_batch = rc.find("span", {"class" : "author_batch"}).string
    if "Pursued" in author_batch :
        college_students += 1
    elif "Batch" in author_batch :
        alumni += 1
    else:
        unknown += 1
print "Percentage of College students :" + str(float(college_students)/float(total_reviews)*100)
print "Percentage of Alumni :" + str(float(alumni)/float(total_reviews)*100)
    

Percentage of College students :90.0
Percentage of Alumni :10.0


### Percentage of "review authors" who are Male/Female

In [114]:
def get_gender(name):
    """
    This function take name of person and returns gender based on api
    https://github.com/appeler/clarifai_gender

    Parameters
    -----------
    name : string
        name of person
    Returns
    --------
    string: gender
    """
    p = 0
    gender = "male"
    for i in range(len(name.split(" "))):
        gender_url = "https://api.genderize.io/?name=" + name.split(" ")[i]
        output = requests.get(gender_url).json()
        if output.get("probability", 0) > p:
            p = output.get("probability", 0)
            gender = output['gender']

    return gender

In [117]:
# We can use already available machine learning solutions to classify names based on learning from existing data of we have any
# 
female = 0
male = 0
total_reviews = len(content)
for rc in content:
    author_name = rc.find("span", {"class" : "author_name"}).string
    print author_name
    gender = get_gender(author_name)
    print gender
    if gender == "male" :
        male += 1
    elif gender == "female" :
        female += 1
    else:
        raise IOError("Invalid name")
print "Percentage of Male :" + str(float(male)/float(total_reviews)*100)
print "Percentage of female :" + str(float(female)/float(total_reviews)*100)
    


Rahul kumar gupta
male
SAURAV SUMAN
male
Durga Bharti
female
Muneet Gill
female
SIDDARTH VETURI
male
Ch sairam hithesh
male
Faiz ul haq
male
Ampolu Venkat Anilkumar
male
Ravi Ranjan Singh
male
BURI MADHU MANEESH
male
Percentage of Male :80.0
Percentage of female :20.0
